### Importing the Libraries

In [17]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import plot_model
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [18]:
tf.__version__ 

'2.12.0'

### Part 1 - Data Preprocessing

#### Generating images for the Training set

In [44]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)


#### Generating images for the Test set

In [20]:
test_datagen = ImageDataGenerator(rescale = 1./255)

### Creating the Training set

In [47]:
training_set = train_datagen.flow_from_directory("/Studies/Year4/Semester8/realtimeASL/Sign-Language-To-Text-Conversion/dataSet/trainingData",                                
                                                 target_size = (128, 128),
                                                 batch_size = 10,
                                                 color_mode = 'grayscale',                                
                                                 class_mode = 'categorical')

Found 12845 images belonging to 27 classes.


In [48]:
validation_set = test_datagen.flow_from_directory('/Studies/Year4/Semester8/realtimeASL/Sign-Language-To-Text-Conversion/dataSet/validationData/',
                                            target_size = (128, 128),                                  
                                            batch_size = 10,        
                                            color_mode = 'grayscale',
                                            class_mode = 'categorical')

Found 3517 images belonging to 27 classes.


### Part 2 - Building the CNN

#### Initializing the CNN

In [49]:
classifier = tf.keras.models.Sequential()

#### Step 1 - Convolution

In [50]:
classifier.add(tf.keras.layers.Conv2D(filters=32,
                                     kernel_size=3, 
                                     padding="same", 
                                     activation="relu", 
                                     input_shape=[128, 128, 1]))

#### Step 2 - Pooling

In [51]:
classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Adding a second convolutional layer

In [52]:
classifier.add(tf.keras.layers.Conv2D(filters=32, 
                                      kernel_size=3, 
                                      padding="same", 
                                      activation="relu"))

classifier.add(tf.keras.layers.MaxPool2D(pool_size=2, 
                                         strides=2, 
                                         padding='valid'))

#### Step 3 - Flattening

In [53]:
classifier.add(tf.keras.layers.Flatten())

#### Step 4 - Full Connection

In [54]:
classifier.add(tf.keras.layers.Dense(units=128, 
                                     activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=96, activation='relu'))
classifier.add(tf.keras.layers.Dropout(0.40))
classifier.add(tf.keras.layers.Dense(units=64, activation='relu'))
classifier.add(tf.keras.layers.Dense(units=27, activation='softmax')) # softmax for more than 2

### Part 3 - Training the CNN

#### Compiling the CNN

In [55]:
classifier.compile(optimizer = 'adam', 
                   loss = 'categorical_crossentropy', 
                   metrics = ['accuracy'])

#### Training the CNN on the Training set and evaluating it on the Test set

In [56]:
classifier.summary()
a = classifier.input_shape
print(a)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 128, 128, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 64, 64, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 64, 64, 32)        9248      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 32, 32, 32)       0         
 2D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 32768)             0         
                                                                 
 dense_8 (Dense)             (None, 128)              

In [57]:
classifier.fit(training_set,
                  epochs = 5,
                  validation_data = validation_set)

Epoch 1/2
1285/1285 [==============================] - 144s 111ms/step - loss: 2.7833 - accuracy: 0.1619 - val_loss: 0.9600 - val_accuracy: 0.6912
Epoch 2/2
1285/1285 [==============================] - 82s 64ms/step - loss: 1.0863 - accuracy: 0.6311 - val_loss: 0.2259 - val_accuracy: 0.9355


Testing the Model

In [58]:
test_data_directory = '/Studies/Year4/Semester8/realtimeASL/Sign-Language-To-Text-Conversion/dataSet/testingData/'

# Load the test dataset using flow_from_directory()
test_set = test_datagen.flow_from_directory(
    test_data_directory,
    target_size = (128, 128),
    batch_size = 10,
    color_mode = 'grayscale',
    class_mode = 'categorical'
)
# Load the test dataset

# Evaluate the model on the test set
test_loss, test_accuracy = classifier.evaluate(test_set)

# Print the test loss and accuracy
print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 792 images belonging to 27 classes.
80/80 [==============================] - 4s 51ms/step - loss: 0.2754 - accuracy: 0.9293
Test Loss: 0.2753981351852417
Test Accuracy: 0.9292929172515869


#### Saving the Model

In [ ]:
model_json = classifier.to_json()
with open("model_new.json", "w") as json_file:
    json_file.write(model_json)
print('Model Saved')
classifier.save_weights('model_new.h5')
print('Weights saved')

Model Saved
Weights saved


In [ ]:
import os
os.environ["PATH"] += os.pathsep + 'D:/graphviz/bin'  # Replace with the path to your Graphviz installation

import pydot
from keras.utils import model_to_dot

# Convert the model to a dot object
dot_model = model_to_dot(classifier, show_shapes=True, show_layer_names=True)
dot_model.set('rankdir', 'LR')  # Set direction of the graph (Left to Right)
dot_model.set('splines', 'ortho')  # Use orthogonal edges
dot_model.set_node_defaults(shape='rectangle', fontsize='12')  # Set node shape and font size

# Get the graph from the dot object
graph = pydot.graph_from_dot_data(dot_model.to_string())[0]

# Save the graph as an image file
graph.write_png('model.png')
